In [ ]:
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np

Unzip for files of png and csv-
uploading the images and the files of car's camera

In [ ]:
!unzip /content/drive/Shareddrives/shared_Maor_Kfir/stop_test2/CognataCamera_ann.zip -d /content/extracted_folder
!unzip /content/drive/Shareddrives/shared_Maor_Kfir/stop_test2/CognataCamera.zip -d /content/extracted_folder
!unzip /content/drive/Shareddrives/shared_Maor_Kfir/stop_test2/CognataCamera_sl.zip -d /content/extracted_folder


Take every second image from the folder

In [ ]:
import os

folder_path_png = "/content/extracted_folder/CognataCamera_png"
folder_path_seg = "/content/extracted_folder/CognataCamera_sl_png"
folder_path_csv = "/content/extracted_folder/CognataCamera_ann"

files = os.listdir(folder_path_png)
# files[0] = files[0][:-3] + "csv"
#files[0]

# Loop through all files in the folder
files_png = os.listdir(folder_path_png)
files_seg = os.listdir(folder_path_seg)
files_csv = os.listdir(folder_path_csv)

for i in range(len(files)):
    # Check if the file is an image (you can modify this check to match your image format)
    if files[i].lower().endswith((".png")):
        # Delete the file if its index is not divisible by 5
        if (i + 1) % 2 != 0:
            file_path_png = os.path.join(folder_path_png, files[i])
            file_path_seg = os.path.join(folder_path_seg, files[i])
            os.remove(file_path_png)
            os.remove(file_path_seg)
            file_csv = files[i][:-3] + "csv"
            file_path_csv = os.path.join(folder_path_csv, file_csv)
            os.remove(file_path_csv)

Creating folders for: "traffic signs", "traffic signs masks", "copy of the original frames"

In [ ]:
import os
# folder for traffic_signs
folder_crops_name = 'traffic_signs'
# Set the path to the new folder in the Colab file system
folder_crops_path = os.path.join('/content/', folder_crops_name)
# Create the new folder
try:
  os.mkdir(folder_crops_path)
  print(f"Folder '{folder_crops_name}' created successfully at '{folder_crops_path}'")
except FileExistsError:
  print(f"Folder '{folder_crops_name}' already exists at '{folder_crops_path}'")

# folder for traffic_signs_seg
folder_crops_seg_name = 'traffic_signs_seg'
# Set the path to the new folder in the Colab file system
folder_crops_seg_path = os.path.join('/content', folder_crops_seg_name)
# Create the new folder
try:
  os.mkdir(folder_crops_seg_path)
  print(f"Folder '{folder_crops_seg_name}' created successfully at '{folder_crops_seg_path}'")
except FileExistsError:
  print(f"Folder '{folder_crops_seg_name}' already exists at '{folder_crops_seg_path}'")

# folder for copy of the frames
folder_copy_name = 'copy'
# Set the path to the new folder in the Colab file system
folder_copy_path = os.path.join('/content', folder_copy_name)
# Create the new folder
try:
  os.mkdir(folder_copy_path)
  print(f"Folder '{folder_copy_name}' created successfully at '{folder_copy_path}'")
except FileExistsError:
  print(f"Folder '{folder_copy_name}' already exists at '{folder_copy_path}'")

Code for cropping the traffic signs and tha masks from the frames



In [ ]:
from numpy.core.fromnumeric import shape
from numpy import double
import shutil

#Copy of the folder of the original images
source_folder="/content/extracted_folder/CognataCamera_png"
destination_folder="/content/extracted_folder/CognataCamera_png_copy"
shutil.copytree(source_folder, destination_folder)

'/content/extracted_folder/CognataCamera_png_copy'

In [ ]:
import os
from numpy.core.fromnumeric import shape
from numpy import double

# set the path of the directory you want to iterate through
folder_path_png= '/content/extracted_folder/CognataCamera_png'
folder_path_png_copy= '/content/extracted_folder/CognataCamera_png_copy'
folder_path_csv= '/content/extracted_folder/CognataCamera_ann'
folder_path_seg= '/content/extracted_folder/CognataCamera_sl_png'

# get a list of all the files and directories in the folder
frames= os.listdir(folder_path_png)
data= os.listdir(folder_path_csv)
seg_frames= os.listdir(folder_path_seg)

# iterate through the frame in the folder and do something with each file
for frame in frames:
  frame_path = os.path.join(folder_path_png, frame)
  frame_path_copy = os.path.join(folder_path_png_copy, frame)
  frame_name= os.path.basename(frame)
  data_name= frame_name[:-3] + "csv"
  data_path= f'/content/extracted_folder/CognataCamera_ann/{data_name}'
  seg_frame_path= f'/content/extracted_folder/CognataCamera_sl_png/{frame_name}'

  data=pd.read_csv(data_path)
  im=cv2.imread(frame_path)
  im_copy=cv2.imread(frame_path_copy)
  im_seg=cv2.imread(seg_frame_path)

  #Creating folder for specific seg_frame
  ######################
  folder_crops_seg_name = f'crops_seg_{frame_name}'
  # Set the path to the new folder in the Colab file system
  folder_crops_seg_path = os.path.join('/content/traffic_signs_seg/', folder_crops_seg_name)
  # Create the new folder
  try:
    os.mkdir(folder_crops_seg_path)
    print(f"Folder '{folder_crops_seg_name}' created successfully at '{folder_crops_seg_path}'")
  except FileExistsError:
    print(f"Folder '{folder_crops_seg_name}' already exists at '{folder_crops_seg_path}'")

  #Creating folder for specific frame
  ######################
  folder_crops_name = f'crops_{frame_name}'
  # Set the path to the new folder in the Colab file system
  folder_crops_path = os.path.join('/content/traffic_signs/', folder_crops_name)
  # Create the new folder
  try:
    os.mkdir(folder_crops_path)
    print(f"Folder '{folder_crops_name}' created successfully at '{folder_crops_path}'")
  except FileExistsError:
    print(f"Folder '{folder_crops_name}' already exists at '{folder_crops_path}'")
  ######################

  ##Loop for the data in the csv file and find the coordinates of the traffic sign in the frame, cut it and create new image with black background
  j=0
  num_img=0
  for j in range(len(data)):
    row=data.loc[j]
    if row['object_class_name']=='TrafficSign':
      string_1=row['bounding_box_3D_frontTopLeft']
      string_2=row['bounding_box_3D_frontBottomRight']
      string_1=(string_1.split('[')[1]).split(']')[0]
      string_2=(string_2.split('[')[1]).split(']')[0]
      nums_1=string_1.split(', ')
      nums_2=string_2.split(', ')
      for i in range(len(nums_1)):
        nums_1[i] = int(float(nums_1[i]))
      for i in range(len(nums_2)):
        nums_2[i] = int(float(nums_2[i]))
      nums_x_min=max(min(nums_1[1], nums_2[1]),0)
      nums_x_max=min(max(nums_1[1], nums_2[1]), 1080)
      nums_y_min=max(min(nums_2[0], nums_1[0]),0)
      nums_y_max=min(max(nums_2[0], nums_1[0]), 1920)
      ########################################################
      #The im_copy is the original frame with the corresponding mask
      im_copy[np.any(im_seg==[128,128,192], axis=-1)]=[28,178,140]
      cv2.imwrite(f'/content/copy/{frame_name}', im_copy)
      ########################################################
      img = im[nums_x_min:nums_x_max, nums_y_min:nums_y_max]
      img_seg=im_seg[nums_x_min:nums_x_max, nums_y_min:nums_y_max]
      img_seg[np.any(img_seg!=[128,128,192], axis=-1)]=[0,0,0]
      img[np.any(img_seg!=[128,128,192], axis=-1)]=[0,0,0]
      img_seg[np.any(img_seg==[128,128,192], axis=-1)]=[28,178,140]
      cv2.imwrite(f'/content/traffic_signs/{folder_crops_name}/img_{num_img}.png', img)
      cv2.imwrite(f'/content/traffic_signs_seg/{folder_crops_seg_name}/img_seg_{num_img}.png', img_seg)
      num_img=num_img+1

Code for deleting imgaes which aren't stop by the German Model-irrelevant for our project but it's for identifying the traffic sings

In [ ]:
from array import array
new_model = tf.keras.models.load_model('/content/Traffic_sign_50.h5')

folders_path="/content/traffic_signs"
folders= os.listdir(folders_path)

for folder in sorted(folders):
  images_path = os.path.join(folders_path, folder)
  images= os.listdir(images_path)
  arr =[]
  for img in sorted(images):
    image = cv2.imread(f'{images_path}/{img}')
    image_fromarray = Image.fromarray(image, 'RGB')
    resize_image = image_fromarray.resize((30, 30))
    arr.append(np.array(resize_image))
    X_test = np.array(arr)

  X_test = X_test/255
  pred = new_model.predict(X_test)
  pred = np.argmax(pred, axis=1)
  print(pred)
  # for i in range(len(pred)):
  #    if pred[i]!=14:
  #      os.remove(f'/{images_path}/img_{i}.png')
  #      num_of_frame=images_path.split('_')
  #      images_path_seg="crops_seg_"+num_of_frame[-1]
  #      os.remove(f'/content/semantic_traffic_signs/{images_path_seg}/img_seg_{i}.png')

Creating folders for the cropped traffic signs and the cropped masks

In [ ]:
import os

directory_name = '/content/traffic_sign_AE'
try:
    os.mkdir(directory_name)
    print(f"Directory '{directory_name}' created successfully")
except FileExistsError:
    print(f"Directory '{directory_name}' already exists")

directory_name = '/content/traffic_sign_AE_seg'
try:
    os.mkdir(directory_name)
    print(f"Directory '{directory_name}' created successfully")
except FileExistsError:
    print(f"Directory '{directory_name}' already exists")

Directory '/content/traffic_sign_AE' created successfully
Directory '/content/traffic_sign_AE_seg' created successfully


Change the names of the images in the folders according to the number of simulation(view file int he drive)

In [ ]:
#for regular
# Move the test images and masks to the test directories
test_files = os.listdir('/content/traffic_signs')
address = '/content/traffic_signs'
for file_name in test_files:
    mask_file_name = file_name.split("_")
    #print(mask_file_name)
    mask_file_name=mask_file_name[0]+"18_"+mask_file_name[1]
    #print(mask_file_name)
    old_name = os.path.join(address, file_name)
    new_name = os.path.join(address, mask_file_name)
    os.rename(old_name, new_name)


In [ ]:
# for seg
# Move the test images and masks to the test directories
test_files = os.listdir('/content/traffic_signs_seg')
address = '/content/traffic_signs_seg'
for file_name in test_files:
    mask_file_name = file_name.split("_")
    #print(mask_file_name)
    mask_file_name=mask_file_name[0]+"18_"+mask_file_name[1]+"_"+mask_file_name[2]
    #print(mask_file_name)
    old_name = os.path.join(address, file_name)
    new_name = os.path.join(address, mask_file_name)
    os.rename(old_name, new_name)


Giving the names of the frame to the image and move it to the folders of AE

In [ ]:
import os

def folder_to_AE(directory, path_of_dest_folder_):
  for folder_name in sorted(os.listdir(directory)):
      flag_stop = 0
      if os.path.isdir(os.path.join(directory, folder_name)):
        frame_directory = os.path.join(directory, folder_name)
        for img_name in sorted(os.listdir(frame_directory)):
          image_path = os.path.join(frame_directory, img_name)
          img = cv2.imread(image_path)
          img_name = folder_name
          if flag_stop == 0:
            cv2.imwrite(f'/{path_of_dest_folder_}/{img_name}', img)
          else:
            new_img_name=img_name[:-4]
            cv2.imwrite(f'/{path_of_dest_folder_}/{new_img_name}_{flag_stop}.png', img)
          flag_stop = flag_stop + 1

directory_signs = '/content/traffic_signs'
directory_sign_seg = '/content/traffic_signs_seg'
path_of_dest_folder_ = "/content/traffic_sign_AE"
path_of_dest_folder_seg = "/content/traffic_sign_AE_seg"
folder_to_AE(directory_signs, path_of_dest_folder_)
folder_to_AE(directory_sign_seg, path_of_dest_folder_seg)

Uploading the folders which we created to the drive in zip

In [ ]:
from google.colab import drive
import shutil

# Define the path to the folder you want to zip and upload
folder_path = "/content/traffic_sign_AE"
folder_path_seg = "/content/traffic_sign_AE_seg"
folder_path2 = "/content/copy"
# Define the name of the zip file you want to create
zip_file_name = "traffic_sign_AE"
zip_file_name_seg = "traffic_sign_AE_seg"
zip_file_name2 = "frame_with_masks"
# Zip the folder
shutil.make_archive(zip_file_name, 'zip', folder_path)
shutil.make_archive(zip_file_name_seg, 'zip', folder_path_seg)
shutil.make_archive(zip_file_name2, 'zip', folder_path2)

In [ ]:
# Copy the zip file to Google Drive
drive_path = "/content/drive/Shareddrives/shared_Maor_Kfir/stop_276_1047"
shutil.copy(zip_file_name + ".zip", drive_path)
shutil.copy(zip_file_name_seg + ".zip", drive_path)
shutil.copy(zip_file_name2 + ".zip", drive_path)

Deleting folders

In [ ]:
import shutil
# shutil.rmtree('/content/traffic_sign_AE')
# shutil.rmtree('/content/traffic_sign_AE_seg')
# shutil.rmtree('/content/traffic_signs_seg')
# shutil.rmtree('/content/traffic_signs')
# shutil.rmtree('/content/copy')
shutil.rmtree('/content/extracted_folder')
# !rm /content/frame_with_masks.zip
# !rm /content/traffic_sign_AE_seg.zip
# !rm /content/traffic_sign_AE.zip
